# Machine Learning Mastery With Python
## Chương 7-8: Tiền xử lý dữ liệu & Lựa chọn đặc trưng


Notebook này hướng dẫn các bước tiền xử lý dữ liệu (chuẩn hóa, chuyển đổi, nhị phân hóa) và lựa chọn đặc trưng (feature selection) để chuẩn bị cho các mô hình Machine Learning.

---

## Mục lục
1. [Chuẩn bị dữ liệu](#chuan-bi)
2. [Rescale dữ liệu về [0,1] (MinMaxScaler)](#rescale)
3. [Chuẩn hóa dữ liệu về phân phối chuẩn (StandardScaler)](#standardize)
4. [Chuẩn hóa vector về độ dài 1 (Normalizer)](#normalize)
5. [Nhị phân hóa dữ liệu (Binarizer)](#binarize)
6. [Lựa chọn đặc trưng đơn biến (SelectKBest)](#univariate)
7. [Loại trừ đặc trưng đệ quy (RFE)](#rfe)
8. [Phân tích thành phần chính (PCA)](#pca)
9. [Đánh giá tầm quan trọng đặc trưng (ExtraTrees)](#importance)
10. [Tổng kết](#tong-ket)

<a id="chuan-bi"></a>
## 1. Chuẩn bị dữ liệu

Ở đây, chúng ta sẽ sử dụng dataset Pima Indians Diabetes (dạng CSV) với 8 thuộc tính đầu vào và 1 nhãn đầu ra (class).

In [ ]:
import pandas as pd
import numpy as np

# Đọc dữ liệu
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv('pima-indians-diabetes.data.csv', names=names)

# Tách X (features) và y (label)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

print("Kích thước X:", X.shape)  # (768, 8)
print("5 dòng đầu của X:")
print(X.head())
print("5 giá trị đầu của y:", y.values[:5])

# X chứa các đặc trưng đầu vào
# y là nhãn (0 hoặc 1)

<a id="rescale"></a>
## 2. Rescale dữ liệu về [0,1] bằng MinMaxScaler

**Mục đích:**  
Một số thuật toán (KNN, SVM, Neural Network,...) nhạy cảm với thang đo của dữ liệu. Rescale đưa mọi thuộc tính về cùng thang [0,1] để so sánh công bằng hơn.

**Công thức:**  
\[
X_{\text{scaled}} = \frac{X - X_{\min}}{X_{\max} - X_{\min}}
\]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)
print("5 dòng đầu sau khi rescale về [0,1]:")
print(X_rescaled[:5])

<a id="standardize"></a>
## 3. Chuẩn hóa dữ liệu về phân phối chuẩn (StandardScaler)

**Mục đích:**  
Standardization giúp dữ liệu có trung bình 0 và độ lệch chuẩn 1. Phù hợp cho các mô hình tuyến tính (Logistic Regression, SVM, KNN...).

**Công thức:**  
\[
X_{\text{standardized}} = \frac{X - \mu}{\sigma}
\]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
print("5 dòng đầu sau khi chuẩn hóa (mean=0, std=1):")
print(X_standardized[:5])
print("Mean sau chuẩn hóa (gần 0):", np.mean(X_standardized, axis=0))
print("Std sau chuẩn hóa (gần 1):", np.std(X_standardized, axis=0))

<a id="normalize"></a>
## 4. Chuẩn hóa vector về độ dài 1 (Normalizer)

**Mục đích:**  
Phù hợp khi bạn quan tâm đến hướng của vector đặc trưng hơn là độ lớn (ví dụ: KNN, Cosine similarity).

**Công thức:**  
\[
\vec{x}_{\text{norm}} = \frac{\vec{x}}{||\vec{x}||}
\]

In [ ]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
X_normalized = scaler.fit_transform(X)
print("5 dòng đầu sau khi normalize (vector có norm=1):")
print(X_normalized[:5])

# Kiểm tra norm của từng vector
print("Norm của 5 dòng đầu:", np.linalg.norm(X_normalized[:5], axis=1))

<a id="binarize"></a>
## 5. Nhị phân hóa dữ liệu (Binarizer)

**Mục đích:**  
Chuyển giá trị lớn hơn threshold thành 1, còn lại là 0. Thường dùng cho dữ liệu đặc biệt hoặc tạo đặc trưng mới.

**Ví dụ:** threshold=5, giá trị >5 thành 1, <=5 thành 0

In [ ]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold=5.0)
X_binarized = binarizer.fit_transform(X)
print("5 dòng đầu sau khi binarize (threshold=5):")
print(X_binarized[:5])

<a id="univariate"></a>
## 6. Lựa chọn đặc trưng đơn biến (SelectKBest với chi2)

**Mục đích:**  
Chọn ra k đặc trưng tốt nhất dựa trên kiểm định thống kê giữa từng thuộc tính và nhãn (ở đây dùng chi2 cho phân loại).

**Lưu ý:**  
- Chi2 chỉ dùng cho dữ liệu không âm (non-negative).
- Nên dùng MinMaxScaler trước nếu dữ liệu có giá trị âm.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Đảm bảo dữ liệu không âm
X_nonneg = MinMaxScaler().fit_transform(X)

test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X_nonneg, y)
print("Điểm chi2 của từng thuộc tính:")
print(fit.scores_)
print("Chọn ra 4 đặc trưng tốt nhất (5 dòng đầu):")
print(fit.transform(X_nonneg)[:5])
print("Các cột được chọn:", np.array(names[:-1])[fit.get_support()])

<a id="rfe"></a>
## 7. Loại trừ đặc trưng đệ quy (RFE)

**Mục đích:**  
RFE loại bỏ dần các đặc trưng kém quan trọng nhất dựa vào mô hình (Logistic Regression, SVM, ...).  
Giúp tìm ra tập đặc trưng tối ưu cho mô hình.

**Cách dùng:**  
- Chọn số đặc trưng muốn giữ lại (n_features_to_select)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
rfe = RFE(model, n_features_to_select=3)
fit = rfe.fit(X, y)
print("Các đặc trưng được chọn (True là chọn):")
print(fit.support_)
print("Thứ tự quan trọng (1 là quan trọng nhất):")
print(fit.ranking_)
print("Tên các cột được chọn:", np.array(names[:-1])[fit.support_])

<a id="pca"></a>
## 8. Phân tích thành phần chính (PCA)

**Mục đích:**  
PCA giúp giảm chiều dữ liệu bằng cách tạo ra các thành phần mới (principal components) giữ lại phần lớn phương sai.  
Có thể dùng để trực quan hóa dữ liệu hoặc giảm nhiễu.

**Lưu ý:**  
- PCA là kỹ thuật không giám sát (unsupervised).

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
fit = pca.fit(X)
print("Tỉ lệ phương sai giải thích bởi mỗi thành phần:", fit.explained_variance_ratio_)
print("Các thành phần chính (principal components):")
print(fit.components_)
print("Dữ liệu sau PCA (5 dòng đầu):")
print(fit.transform(X)[:5])

<a id="importance"></a>
## 9. Đánh giá tầm quan trọng đặc trưng (ExtraTrees)

**Mục đích:**  
Dùng mô hình cây (ExtraTrees) để đánh giá mức độ quan trọng của từng thuộc tính với dự đoán nhãn.  
Thường dùng để loại bỏ đặc trưng kém quan trọng.

**Lưu ý:**  
- Kết quả là một vector độ quan trọng (feature_importances_)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X, y)
print("Độ quan trọng của từng đặc trưng:")
for name, score in zip(names[:-1], model.feature_importances_):
    print(f"{name}: {score:.3f}")

<a id="tong-ket"></a>
## 10. Tổng kết

- Đã thực hiện các bước tiền xử lý dữ liệu: rescale, standardize, normalize, binarize.
- Đã thử nghiệm nhiều kỹ thuật lựa chọn đặc trưng: SelectKBest, RFE, PCA, ExtraTrees.
- Việc tiền xử lý và chọn đặc trưng giúp mô hình học máy hiệu quả, chính xác và nhanh hơn.